## Práctico 2
En este práctico se aplicarán los conocimientos adquiridos de redes avanzadas. Basándose en lo aprendido con el práctico 1, esperamos que apliquen la misma metodología en la creación de un pipeline de trabajo para el entrenamiento y evaluación de modelos de redes neuronales avanzados en tareas que les conciernan.

El problema a elegir quedará a su criterio. Algunas posibilidades son:

#### Tareas de PLN (pueden aplicarse a redes convolucionales o recurrentes):
    Análisis de sentimiento sobre los datos del práctico 1.
    Análisis de sentimiento sobre el conjunto de datos de IMDB (disponible en keras.datasets).
    Clasificación de textos del 20 Newsgroup.

#### Tareas de imágenes:
    Reconocimiento de imágenes del CIFAR-10 (disponible en keras.datasets) usando convolucionales.
    Análisis de videos de YouTube usando recurrentes.

Estos son solo algunos ejemplos. Pueden explorar otros conjuntos de datos y ver cómo aplicar las distintas redes (o incluso combinaciones de las mismas). Si van a trabajar en algo de imágenes busquen clasificación de imágenes a color para aprovechas los canales en las redes convolucionales. A la hora de entregar sus trabajos dejen un link al conjunto de datos trabajado.


# Vamos realizar  análisis de sentimiento sobre el conjunto de datos de IMDB (disponible en keras.datasets).



In [20]:
import keras.datasets
from keras.datasets import imdb
from keras.preprocessing import sequence 

from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, Embedding, Average, Lambda, LSTM, Flatten
from keras import optimizers, regularizers
from numpy import array

from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
import numpy


### Paso 1: leer los datos

In [2]:
#download the data
top_words = 88584
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words) #, skip_top=20)

In [3]:
word_to_id = keras.datasets.imdb.get_word_index()
id_to_word = {value:key for key,value in word_to_id.items()}
print(' '.join(id_to_word[id] for id in X_train[2]))

the as there in at by br of sure many br of proving no only women was than doesn't as you never of hat night that with ignored they bad out superman plays of how star so stories film comes defense date of wide they don't do that had with of hollywood br of my seeing fan this of pop out body shots in having because cause it's stick passing first were enjoys for from look seven sense from me superimposition die in character as cuban issues but is you that isn't one song just is him less are strongly not are you that different just even by this of you there is eight when it part are film's love film's 80's was big also light don't wrangling as it in character looked cinematography so stories is far br man acting


In [4]:
len(word_to_id)


88584

### Paso 2: preprocesar los datos

In [5]:
# Truncate and pad the review sequences 
max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length) 
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length) 

In [6]:
#reverse lookup
word_to_id = keras.datasets.imdb.get_word_index()
X_train[2]
#INDEX_FROM=
# word_to_id = {k:(v+INDEX_FROM) for k,v in word_to_id.items()}
word_to_id["<PAD>"] = 0
word_to_id["<START>"] = 1
word_to_id["<UNK>"] = 2
id_to_word = {value:key for key,value in word_to_id.items()}
print(' '.join(id_to_word[id] for id in X_train[2] ))

<PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD

### Paso 3: crea el modelo

### Model1 con recurrent layer


In [8]:
# Build the model 
embedding_vector_length = 32 
model = Sequential() 
model.add(Embedding(top_words, embedding_vector_length, input_length=max_review_length)) 
model.add(LSTM(100)) 
model.add(Dense(1, activation='sigmoid')) 
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy']) 
print(model.summary()) 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 500, 32)           2834688   
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 2,887,989
Trainable params: 2,887,989
Non-trainable params: 0
_________________________________________________________________
None


In [10]:
model.layers[0].get_config()

{'activity_regularizer': None,
 'batch_input_shape': (None, 500),
 'dtype': 'float32',
 'embeddings_constraint': None,
 'embeddings_initializer': {'class_name': 'RandomUniform',
  'config': {'maxval': 0.05, 'minval': -0.05, 'seed': None}},
 'embeddings_regularizer': None,
 'input_dim': 88584,
 'input_length': 500,
 'mask_zero': False,
 'name': 'embedding_2',
 'output_dim': 32,
 'trainable': True}

In [11]:
model.layers[1].get_config()

{'activation': 'tanh',
 'activity_regularizer': None,
 'bias_constraint': None,
 'bias_initializer': {'class_name': 'Zeros', 'config': {}},
 'bias_regularizer': None,
 'dropout': 0.0,
 'go_backwards': False,
 'implementation': 1,
 'kernel_constraint': None,
 'kernel_initializer': {'class_name': 'VarianceScaling',
  'config': {'distribution': 'uniform',
   'mode': 'fan_avg',
   'scale': 1.0,
   'seed': None}},
 'kernel_regularizer': None,
 'name': 'lstm_1',
 'recurrent_activation': 'hard_sigmoid',
 'recurrent_constraint': None,
 'recurrent_dropout': 0.0,
 'recurrent_initializer': {'class_name': 'Orthogonal',
  'config': {'gain': 1.0, 'seed': None}},
 'recurrent_regularizer': None,
 'return_sequences': False,
 'return_state': False,
 'stateful': False,
 'trainable': True,
 'unit_forget_bias': True,
 'units': 100,
 'unroll': False,
 'use_bias': True}

In [15]:
model.layers[2].get_config()

{'activation': 'sigmoid',
 'activity_regularizer': None,
 'bias_constraint': None,
 'bias_initializer': {'class_name': 'Zeros', 'config': {}},
 'bias_regularizer': None,
 'kernel_constraint': None,
 'kernel_initializer': {'class_name': 'VarianceScaling',
  'config': {'distribution': 'uniform',
   'mode': 'fan_avg',
   'scale': 1.0,
   'seed': None}},
 'kernel_regularizer': None,
 'name': 'dense_2',
 'trainable': True,
 'units': 1,
 'use_bias': True}

In [9]:
#Train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=3, batch_size=64) 

/users/fpalacios/miniconda3/envs/diplodatos/lib/python3.5/site-packages/ipykernel/__main__.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  from ipykernel import kernelapp as app


Train on 25000 samples, validate on 25000 samples
Epoch 1/3
25000/25000 [==============================] - 251s 10ms/step - loss: 0.4625 - acc: 0.7769 - val_loss: 0.3204 - val_acc: 0.8694
Epoch 2/3
25000/25000 [==============================] - 248s 10ms/step - loss: 0.2382 - acc: 0.9090 - val_loss: 0.3197 - val_acc: 0.8697
Epoch 3/3
25000/25000 [==============================] - 248s 10ms/step - loss: 0.1173 - acc: 0.9594 - val_loss: 0.3924 - val_acc: 0.8658


In [13]:
model.save('Practico2_IMDB_recurrent.h5')

In [14]:
#Evaluate the model
scores = model.evaluate(X_test, y_test, verbose=0) 
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 86.58%


### Paso 6: Ejemplos de predicciones

In [16]:
y_test[0]

0

In [17]:
y_test[1]

1

In [18]:
neg = (' '.join(id_to_word[id] for id in X_test[0] ))
pos = (' '.join(id_to_word[id] for id in X_test[1] ))

review=pos
tmp=[]
for word in review.split(" "):
    try:
        tmp.append(word_to_id[word])
    except:
        print('Esta palabra no está en el corpus: ',word)

    tmp_padded = sequence.pad_sequences([tmp], maxlen=max_review_length) 
pred = model.predict(array([tmp_padded][0]))

print('Positiva: ',review,pred)
print('')

review=neg
tmp=[]
for word in review.split(" "):
    try:
        tmp.append(word_to_id[word])
    except:
        print('Esta palabra no está en el corpus: ',word)

    tmp_padded = sequence.pad_sequences([tmp], maxlen=max_review_length) 
pred = model.predict(array([tmp_padded][0]))
print('Negativa: ', review,pred)



Positiva:  <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

### Model2 con convolutional layer

In [21]:
seed = 7
numpy.random.seed(seed)

max_words = 500

model2 = Sequential()
model2.add(Embedding(top_words, 32, input_length=max_words))
model2.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model2.add(MaxPooling1D(pool_size=2))
model2.add(Flatten())
model2.add(Dense(250, activation='relu'))
model2.add(Dense(1, activation='sigmoid'))
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model2.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 500, 32)           2834688   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 500, 32)           3104      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 250, 32)           0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 8000)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 250)               2000250   
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 251       
Total params: 4,838,293
Trainable params: 4,838,293
Non-trainable params: 0
_________________________________________________________________


In [22]:
model2.layers[1].get_config()

{'activation': 'relu',
 'activity_regularizer': None,
 'bias_constraint': None,
 'bias_initializer': {'class_name': 'Zeros', 'config': {}},
 'bias_regularizer': None,
 'data_format': 'channels_last',
 'dilation_rate': (1,),
 'filters': 32,
 'kernel_constraint': None,
 'kernel_initializer': {'class_name': 'VarianceScaling',
  'config': {'distribution': 'uniform',
   'mode': 'fan_avg',
   'scale': 1.0,
   'seed': None}},
 'kernel_regularizer': None,
 'kernel_size': (3,),
 'name': 'conv1d_1',
 'padding': 'same',
 'strides': (1,),
 'trainable': True,
 'use_bias': True}

In [23]:
# Fit the model
model2.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=2, batch_size=128, verbose=2)
# Final evaluation of the model
scores = model2.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
 - 12s - loss: 0.4239 - acc: 0.7828 - val_loss: 0.2747 - val_acc: 0.8860
Epoch 2/2
 - 12s - loss: 0.1317 - acc: 0.9530 - val_loss: 0.3378 - val_acc: 0.8700
Accuracy: 87.00%


In [24]:
model2.save('Practico2_IMDB_convolutional.h5')

### Ejemplos de predicciones

In [25]:
neg = (' '.join(id_to_word[id] for id in X_test[0] ))
pos = (' '.join(id_to_word[id] for id in X_test[1] ))

review=pos
tmp=[]
for word in review.split(" "):
    try:
        tmp.append(word_to_id[word])
    except:
        print('Esta palabra no está en el corpus: ',word)

    tmp_padded = sequence.pad_sequences([tmp], maxlen=max_review_length) 
pred = model.predict(array([tmp_padded][0]))

print('Positiva: ',review,pred)
print('')

review=neg
tmp=[]
for word in review.split(" "):
    try:
        tmp.append(word_to_id[word])
    except:
        print('Esta palabra no está en el corpus: ',word)

    tmp_padded = sequence.pad_sequences([tmp], maxlen=max_review_length) 
pred = model2.predict(array([tmp_padded][0]))
print('Negativa: ', review,pred)


Positiva:  <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>